In [1]:
#Web Scrapping to collect the data from Magicbricks 

import re
import requests
from bs4 import BeautifulSoup
import urllib

url='https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&cityName=Mumbai&page=1'

r=requests.get(url)
c=r.content
soup=BeautifulSoup(c,'html.parser')
page_no=soup.find("a",{"class":"toc"}).previous_sibling.text.strip()

print(page_no)
page_no=int(page_no)
l=[]
for i in range(1,21):
    #print('https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&cityName=Mumbai&page=%d'%i)
    page_url='https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&cityName=Mumbai&page=%d'%i
    r=requests.get(page_url)
    c=r.content
    soup=BeautifulSoup(c,'html.parser')
    data=soup.findAll("div",{"class":"flex relative clearfix m-srp-card__container"})
    #print(data)
    for each in data:
        info={}
        a=each.find("a",{"class":"m-srp-card__title"}).text.replace("\n"," ")
        info['city']=a[a.find("in")+2:].strip()
        info['price']=each.find("span",{"class":"m-srp-card__price"}).text.strip()
        info['bhk']=each.find("span",{"class":"m-srp-card__title__bhk"}).text.strip()
        p=each.find_all("div",{"class":"m-srp-card__summary__item"})
        for i in p:
            title=i.find("div",{"class":"m-srp-card__summary__title"}).text.strip()
            info[title]=i.find("div",{"class":"m-srp-card__summary__info"}).text.strip()

        l.append(info)

1955


In [2]:
#Data Collected After Scrapping
print(l)

[{'city': 'Thakur Village, Kandivali East', 'price': '90 Lac', 'bhk': '1 BHK Flat/Apartment', 'carpet area': '410\xa0sqft', 'status': "Possession by Dec '21", 'floor': '18 out of 50 floors', 'transaction': 'New Property', 'furnishing': 'Unfurnished', 'overlooking': 'Garden/Park, Main Road', 'car parking': '1 Covered', 'bathroom': '2', 'balcony': '2', 'ownership': 'Freehold'}, {'city': 'Chembur', 'price': '3.76 Cr', 'bhk': '3 BHK Flat/Apartment', 'carpet area': '1055\xa0sqft', 'status': 'Ready to Move', 'floor': '4 out of 14 floors', 'transaction': 'New Property', 'furnishing': 'Unfurnished', 'society': 'Golf Scappe', 'car parking': '1 Covered', 'bathroom': '3'}, {'city': 'Matunga East', 'price': '1.49 Cr', 'bhk': '2 BHK Flat/Apartment', 'carpet area': '33\xa0sqm', 'transaction': 'New Property', 'furnishing': 'Unfurnished', 'society': 'Hubtown Harmony', 'bathroom': '2'}, {'city': 'Prabhadevi', 'price': '4.25 Cr', 'bhk': '2 BHK Flat/Apartment', 'super area': '1007\xa0sqft', 'status': "Po

In [3]:
#Store the data in DataFrame and Data Cleaning

import numpy as np
import pandas as pd
df=pd.DataFrame(l)
df['bhk']=df['bhk'].str.replace('BHK Flat/Apartment','')
#df['car parking']=df['car parking'].str.replace('Covered','')
#df['car parking']=df['car parking'].str.replace('1 Open','1')
new = df["car parking"].str.split(" ", n = 1, expand = True) 
#print(new)
# making seperate first name column from new data frame 
df["no of parking"]= new[0] 
# making seperate last name column from new data frame 
df["Parking type"]= new[1]
df.head()


df['carpet area']=df['carpet area'].str.replace('sqft','')
#df.head()
df['carpet area']=df['carpet area'].str.replace('sqm','')
df.head()

type(df['carpet area'])

df['carpet area'] = df['carpet area'].astype(np.float64)
df['carpet area'].dtype
df['carpet area']=np.where(df['carpet area']<100,df['carpet area']*10.7639,df['carpet area'])
df['carpet area']=round(df['carpet area'],0)
### replace NaN of balcony with zero
df['balcony']=df['balcony'].fillna(0)
df.head

#### replace NaN of bathroom with zero
df['bathroom']=df["bathroom"].fillna(0)
df.head

new = df["car parking"].str.split(" ", n = 1, expand = True) 
new
#df['no of parking']=df["no of parking"].fillna(0)

#df['floor'].replace(" ",0).bfill()
df['floor'] = df['floor'].apply(lambda x: 0 if x == ' ' else x)
df.head
df['floor']=df["floor"].fillna(0)
df.head


<bound method NDFrame.head of     balcony bathroom               bhk        car parking  carpet area  \
0         2        2                1           1 Covered        410.0   
1         0        3                3           1 Covered       1055.0   
2         0        2                2                 NaN        355.0   
3         0        2                2                 NaN          NaN   
4         0        2                2           1 Covered        938.0   
5         0        2                2                 NaN        479.0   
6         0        0  Studio Apartment                NaN        281.0   
7         0        2                2           1 Covered        848.0   
8         0        2                2           1 Covered        625.0   
9         0        3                3           1 Covered       1030.0   
10        0        2                2                 NaN          NaN   
11        0        4                4                 NaN          NaN   
12      

In [4]:
#Write the data to excel
print(df)
with pd.ExcelWriter('Project_file.xlsx') as writer:
         df.to_excel(writer,sheet_name='Properties',index=False)

    balcony bathroom               bhk        car parking  carpet area  \
0         2        2                1           1 Covered        410.0   
1         0        3                3           1 Covered       1055.0   
2         0        2                2                 NaN        355.0   
3         0        2                2                 NaN          NaN   
4         0        2                2           1 Covered        938.0   
5         0        2                2                 NaN        479.0   
6         0        0  Studio Apartment                NaN        281.0   
7         0        2                2           1 Covered        848.0   
8         0        2                2           1 Covered        625.0   
9         0        3                3           1 Covered       1030.0   
10        0        2                2                 NaN          NaN   
11        0        4                4                 NaN          NaN   
12        0        3                3 

In [5]:
#Read The data from Excel
def convert_cp(cell):
    if cell == "":
        return 'Not Covered'
    else:
        return cell
    
def convert_facing(cell):
    if cell == "":
        return 'No Information'
    else:
        return cell

def convert_overlooking(cell):
    if cell == "":
        return 'No Information'
    else:
        return cell
    
def convert_ownership(cell):
    if cell == "":
        return 'Unknown'
    else:
        return cell

def convert_parking(cell):
    if cell == "":
        return '0'
    else:
        return cell
    
    
df_0 = pd.read_excel("Project_File.xlsx",sheet_name='Properties',na_values =['NA',' ','Not Available','?'],converters = {'car parking':convert_cp,'facing':convert_facing,'overlooking':convert_overlooking,
                                                                                                                        'ownership':convert_ownership,'no of parking':convert_parking,'Parking type':convert_cp})

print(type(df_0))

<class 'pandas.core.frame.DataFrame'>


In [6]:
df_0

,balcony,bathroom,bhk,car parking,carpet area,city,dimensions,facing,floor,furnishing,overlooking,ownership,price,society,status,super area,transaction,no of parking,Parking type
0,2,2,1,1 Covered,410.0,"Thakur Village, Kandivali East",NaN,No Information,18 out of 50 floors,Unfurnished,"Garden/Park, Main Road",Freehold,90 Lac,NaN,Possession by Dec '21,NaN,New Property,1,Covered
1,0,3,3,1 Covered,1055.0,Chembur,NaN,No Information,4 out of 14 floors,Unfurnished,No Information,Unknown,3.76 Cr,Golf Scappe,Ready to Move,NaN,New Property,1,Covered
2,0,2,2,Not Covered,355.0,Matunga East,NaN,No Information,0,Unfurnished,No Information,Unknown,1.49 Cr,Hubtown Harmony,NaN,NaN,New Property,0,Not Covered
3,0,2,2,Not Covered,NaN,Prabhadevi,NaN,No Information,0,Unfurnished,No Information,Unknown,4.25 Cr,Ajmera Classique,Possession by Dec '22,1007 sqft,New Property,0,Not Covered
4,0,2,2,1 Covered,938.0,Sion,NaN,No Information,0,Semi-Furnished,No Information,Unknown,3.77 Cr,Beaumonte,Possession by Dec '23,NaN,New Property,1,Covered
5,0,2,2,Not Covered,479.0,Kandivali West,NaN,No Information,0,Unfurnished,No Information,Unknown,1.22 Cr,Siddha Seabrook,NaN,NaN,New Property,0,Not Covered
6,0,0,Studio Apartment,Not Covered,281.0,"UK Iridium, Kandivali East",NaN,No Information,0,Unfurnished,No Information,Unknown,53.7 Lac,UK Iridium,NaN,NaN,New Property,0,Not Covered
7,0,2,2,1 Covered,848.0,Malad East,NaN,No Information,10 out of 55 floors,Unfurnished,No Information,Unknown,2.09 Cr,Omkar Alta Monte,Possession by Dec '23,NaN,New Property,1,Covered
8,0,2,2,1 Covered,625.0,Goregaon West,NaN,No Information,1 out of 22 floors,Semi-Furnished,No Information,Freehold,1.80 Cr,Saanvi Heights,Ready to Move,NaN,New Property,1,Covered
9,0,3,3,1 Covered,1030.0,Jogeshwari East,NaN,No Information,9 out of 14 floors,Unfurnished,No Information,Unknown,3.20 Cr,Shubham Lily White,Ready to Move,NaN,New Property,1,Covered


In [7]:
#Cleaning of the BHK Column

#Cleaning of the BHK Column
bhk =df_0['bhk']
#bhk[0]
for i in range(0,len(bhk)):
    bhk[i]=re.sub("\D", "",bhk[i])
bhk=bhk.replace('','1',regex=True)
bhk=bhk.astype(float)
df_0['bhk']=bhk


C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [8]:
#Pseudo Labels for Categorical Variables

import scipy 
from scipy.stats.stats import pearsonr
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#df_0['carpet area']=int(df_0['carpet area'])
#print(df_0.corr(method='pearson'))
#print(pearsonr(df_0['carpet area'],df_0['balcony']))
print(type(df_0['carpet area']))
print(type(df_0['balcony']))
print(type(df_0['ownership']))


le = preprocessing.LabelEncoder()
df_cat = df_0[['car parking','facing','furnishing','overlooking','ownership','transaction','Parking type']]

#df_cat['bhk']=le.fit_transform(df_cat['bhk'].astype(str))
df_cat['car parking']=le.fit_transform(df_cat['car parking'].astype(str))
df_cat['facing']=le.fit_transform(df_cat['facing'].astype(str))
#df_cat['floor']=le.fit_transform(df_cat['floor'].astype(str))
df_cat['furnishing']=le.fit_transform(df_cat['furnishing'].astype(str))
df_cat['overlooking']=le.fit_transform(df_cat['overlooking'].astype(str))
df_cat['ownership']=le.fit_transform(df_cat['ownership'].astype(str))
df_cat['transaction']=le.fit_transform(df_cat['transaction'].astype(str))
df_cat['Parking type']=le.fit_transform(df_cat['Parking type'].astype(str))

print(df_cat.head())

df_cat1=df_cat.apply(LabelEncoder().fit_transform)
df_cat1.head()

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

   car parking  facing  furnishing  overlooking  ownership  transaction  \
0            0       1           2            1          1            0   
1            0       1           2            7          4            0   
2           13       1           2            7          4            0   
3           13       1           2            7          4            0   
4            0       1           1            7          4            0   

   Parking type  
0             0  
1             0  
2             3  
3             3  
4             0  


C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prach\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,car parking,facing,furnishing,overlooking,ownership,transaction,Parking type
0,0,1,2,1,1,0,0
1,0,1,2,7,4,0,0
2,13,1,2,7,4,0,3
3,13,1,2,7,4,0,3
4,0,1,1,7,4,0,0


In [9]:
#Data Cleaning for the Price Column
price = df['price']
price
price=price.replace('Lac','',regex=True)
price=price.replace('Cr','',regex=True).astype(float)
price

for i in range(0,len(price)):
    if price[i]<10:
        price[i]=price[i]*100
df_0['price in lakhs']=price
print(df_0)
#print(price)



#print(df['price'])
#df_0['floor']=floor_n


     balcony  bathroom  bhk        car parking  carpet area  \
0          2         2  1.0          1 Covered        410.0   
1          0         3  3.0          1 Covered       1055.0   
2          0         2  2.0        Not Covered        355.0   
3          0         2  2.0        Not Covered          NaN   
4          0         2  2.0          1 Covered        938.0   
5          0         2  2.0        Not Covered        479.0   
6          0         0  1.0        Not Covered        281.0   
7          0         2  2.0          1 Covered        848.0   
8          0         2  2.0          1 Covered        625.0   
9          0         3  3.0          1 Covered       1030.0   
10         0         2  2.0        Not Covered          NaN   
11         0         4  4.0        Not Covered          NaN   
12         0         3  3.0        Not Covered       1645.0   
13         0         2  2.0          1 Covered        752.0   
14         0         2  2.0        Not Covered        6

In [10]:
#Data Cleaning for the FLoor Column
floor_n=df_0['floor']
floor_n =df_0['floor'].str[:2]

floor_n=floor_n.astype(str)


floor_n=floor_n.replace('Lo','0',regex=True)
floor_n=floor_n.replace('Up','0',regex=True)
floor_n=floor_n.replace('Gr','0',regex=True)


floor_n=floor_n.replace('nan','0',regex=True)


floor_n=floor_n.astype(float)
floor_n

df_0['floor']=floor_n
print(df_0['floor'])

0      18.0
1       4.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7      10.0
8       1.0
9       9.0
10      0.0
11     11.0
12      0.0
13      0.0
14      0.0
15      9.0
16     12.0
17      5.0
18      0.0
19      0.0
20      0.0
21      0.0
22      4.0
23      0.0
24      4.0
25      0.0
26      0.0
27      0.0
28      0.0
29      1.0
       ... 
571     4.0
572     4.0
573     0.0
574    24.0
575    20.0
576    11.0
577     2.0
578    19.0
579     1.0
580     0.0
581     5.0
582    10.0
583     1.0
584     3.0
585     9.0
586     1.0
587     1.0
588     3.0
589     2.0
590    31.0
591    30.0
592    12.0
593    30.0
594    10.0
595     7.0
596     8.0
597     1.0
598     4.0
599     0.0
600     3.0
Name: floor, Length: 601, dtype: float64


In [11]:
#Data Cleaning for The area column
sa = df['super area']
sa = sa.replace('sqft','',regex=True)

sa=sa.replace('sqm','',regex=True)
sa=sa.astype(str)
sa=sa.replace('nan','0',regex=True)
sa=sa.astype(float)
sa

0         0.0
1         0.0
2         0.0
3      1007.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10      892.0
11     1407.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18      994.0
19        0.0
20        0.0
21     1580.0
22      987.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28      640.0
29      780.0
        ...  
571       0.0
572       0.0
573       0.0
574       0.0
575       0.0
576       0.0
577       0.0
578       0.0
579       0.0
580     700.0
581       0.0
582       0.0
583       0.0
584       0.0
585       0.0
586       0.0
587     700.0
588       0.0
589     830.0
590       0.0
591       0.0
592       0.0
593       0.0
594       0.0
595       0.0
596       0.0
597       0.0
598       0.0
599       0.0
600       0.0
Name: super area, Length: 601, dtype: float64

In [12]:
ca = df['carpet area']
#ca=ca.replace('NaN','0',regex=True)
ca = ca.fillna(0)
ca = ca.astype(float)

ta=ca+sa

# We have created a new column for the area

df_0['total area']=ta
#print(ta.head(7))
print(df_0.head(7))
#print(ca)'''

   balcony  bathroom  bhk  car parking  carpet area  \
0        2         2  1.0    1 Covered        410.0   
1        0         3  3.0    1 Covered       1055.0   
2        0         2  2.0  Not Covered        355.0   
3        0         2  2.0  Not Covered          NaN   
4        0         2  2.0    1 Covered        938.0   
5        0         2  2.0  Not Covered        479.0   
6        0         0  1.0  Not Covered        281.0   

                             city dimensions          facing  floor  \
0  Thakur Village, Kandivali East        NaN  No Information   18.0   
1                         Chembur        NaN  No Information    4.0   
2                    Matunga East        NaN  No Information    0.0   
3                      Prabhadevi        NaN  No Information    0.0   
4                            Sion        NaN  No Information    0.0   
5                  Kandivali West        NaN  No Information    0.0   
6      UK Iridium, Kandivali East        NaN  No Information  

In [13]:
#data cleaning for the Bathroom Column
bath = df['bathroom']
bath = bath.replace('> 10','10',regex=True)
df_0['bathroom']=bath
df_0['bathroom']=df_0['bathroom'].astype(int)

In [14]:
#print(df)
#We are writing the completely clean Dataset to excel to use it for EDA 
with pd.ExcelWriter('Clean_DF.xlsx') as writer:
         df_0.to_excel(writer,sheet_name='Properties',index=False)

In [15]:

df_0['no of parking'] = df_0['no of parking'].astype(int)
print(df_0.dtypes)

balcony             int64
bathroom            int32
bhk               float64
car parking        object
carpet area       float64
city               object
dimensions         object
facing             object
floor             float64
furnishing         object
overlooking        object
ownership          object
price              object
society            object
status             object
super area         object
transaction        object
no of parking       int32
Parking type       object
price in lakhs    float64
total area        float64
dtype: object


In [16]:
#print(df_0.corr(method='pearson'))
#Combining the numerical features and categorical features into a new dataframe
df_num = df_0[['bhk','balcony','bathroom','total area','floor','no of parking','price in lakhs']]
new_df=pd.concat([df_cat1,df_num],axis=1)
new_df.head(7)
print(new_df.corr(method='pearson'))
new_df['no of parking']=new_df['no of parking'].astype(int)
new_df.head()

                car parking    facing  furnishing  overlooking  ownership  \
car parking        1.000000  0.041107    0.005507     0.139358   0.114006   
facing             0.041107  1.000000   -0.106091    -0.068801  -0.154764   
furnishing         0.005507 -0.106091    1.000000     0.020163   0.138842   
overlooking        0.139358 -0.068801    0.020163     1.000000   0.434706   
ownership          0.114006 -0.154764    0.138842     0.434706   1.000000   
transaction        0.181160  0.209193   -0.341535    -0.061137  -0.291766   
Parking type       0.600610  0.043556   -0.053945     0.016495  -0.006818   
bhk                0.099387  0.128278   -0.108801    -0.050166  -0.059399   
balcony            0.076967  0.059484   -0.048887    -0.190126  -0.147460   
bathroom           0.138477  0.114041   -0.080398     0.006555  -0.024546   
total area         0.131736  0.080161   -0.109635    -0.022494  -0.026813   
floor             -0.016520 -0.038064    0.103910    -0.031264  -0.081091   

,car parking,facing,furnishing,overlooking,ownership,transaction,Parking type,bhk,balcony,bathroom,total area,floor,no of parking,price in lakhs
0,0,1,2,1,1,0,0,1.0,2,2,410.0,18.0,1,90.0
1,0,1,2,7,4,0,0,3.0,0,3,1055.0,4.0,1,376.0
2,13,1,2,7,4,0,3,2.0,0,2,355.0,0.0,0,149.0
3,13,1,2,7,4,0,3,2.0,0,2,1007.0,0.0,0,425.0
4,0,1,1,7,4,0,0,2.0,0,2,938.0,0.0,1,377.0


In [17]:

#Model Building

from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import scale
features = new_df.loc[:,'car parking':'no of parking']
price = new_df.loc[:,'price in lakhs']

#print(features.corr())
features = features.drop(["car parking","furnishing", "overlooking", "ownership","Parking type","transaction","facing"], axis=1)

#feature_s = scale(features)
print(features.head())

feat_train,feat_test,price_train,price_test= train_test_split(features,price,test_size=0.25,random_state=80)

#print(np.where(np.isnan(new_df)))
linR = LinearRegression().fit(feat_train,price_train)

pred = linR.predict(feat_test)

print(linR.score(feat_test,price_test))

#print(df)
print(pred)

   bhk  balcony  bathroom  total area  floor  no of parking
0  1.0        2         2       410.0   18.0              1
1  3.0        0         3      1055.0    4.0              1
2  2.0        0         2       355.0    0.0              0
3  2.0        0         2      1007.0    0.0              0
4  2.0        0         2       938.0    0.0              1
0.36226753573507153
[311.72356733 220.61218921 316.50463248 348.93149912 506.10707612
 218.77918797 207.93127117  68.55162004  34.72400896 374.16866134
 410.71266121 205.76189283 353.66233816 502.59506329 321.48112328
 325.28404679 194.85341112 349.31722751 434.76036189 217.34881092
 247.28220442 214.10674731 218.77632869 220.62310649 175.64993184
  64.78589747 205.76709153 112.61921932 175.17634536 205.65973831
 306.64200411 203.27754583 222.53244163 216.91239516 369.94641299
 214.12026394 225.72104563 348.35185908  76.04703718 172.76838292
 212.20495029 110.3126316  108.00708362 339.33458983 353.70288804
 340.80830329  73.76754134

C:\Users\prach\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
#UI for the Prediction System

import tkinter
from tkinter import *
top = tkinter.Tk()
#from pandastable import Table, TableModel
# Code to add widgets will go here...
top.geometry("500x600")

def retrieve_input():
    bhk=float(E1.get())
    balcony=int(E2.get())
    bathroom=int(E3.get())
    tot_area=float(E4.get())
    floor=float(E5.get())
    parking=int(E6.get())
    df_val=pd.DataFrame([{"bhk":bhk,"balcony":balcony,"bathroom":bathroom,"total area":tot_area,"floor":floor,"no of parking":parking}],
                        columns = [ 'bhk', 'balcony' ,'bathroom','total area','floor','no of parking'])
    print("Data Frame Created")
    #Label(top,text=df_val).pack()
    #s_df_val = scale(df_val)
    pred_val = linR.predict(df_val)
    print(pred_val)
    #pv=str(pred_val)
    pv=float(pred_val)
    pv=round(pv)
    message="The Predicted Value of This Property is "+"Rs. "+str(pv) +"Lakhs"
    #Label(top,text="The Prediction is ").pack()
    #pt = Table(f, dataframe=df_val,showtoolbar=True, showstatusbar=True)
    #pt.show()
    Label(top,text=message).pack()
    
    
L1 = Label(top,text="BHK")
L1.pack()
E1 = Entry(bd =5)
E1.pack()

L2 = Label(top,text="Balcony")
L2.pack()
E2 = Entry(bd =5)
E2.pack()

L3 = Label(top,text="Bathroom")
L3.pack()
E3 = Entry(bd =5)
E3.pack()

L4 = Label(top,text="Total Area")
L4.pack()
E4 = Entry(bd =5)
E4.pack()

L5 = Label(top,text="No. of Floor")
L5.pack()
E5 = Entry(bd =5)
E5.pack()


L6 = Label(top,text="No. of Parking")
L6.pack()
E6 = Entry(bd =5)
E6.pack()

button=Button(top,height=1,width=10,text="Run the Model",command=lambda:retrieve_input())
button.pack()
top.mainloop()

Data Frame Created
[343.04370516]
